In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as tf
from tqdm import tqdm
from predict import *

#in HIST2ST, collections.Iterable is imported via 
# from HIST2ST import * --> from transformer import* 
# --> from easydl import * --> from .pytorch import *
# from .utils import * --> from collections import Iterable.
# the alias Iterable has been deprecated however,
# so you need a workaround: second answer: https://stackoverflow.com/questions/72032032/importerror-cannot-import-name-iterable-from-collections-in-python
import collections.abc
collections.Iterable = collections.abc.Iterable


from HIST2ST import *
from dataset import ViT_HER2ST, ViT_SKIN
from scipy.stats import pearsonr,spearmanr
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from copy import deepcopy as dcp
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score

import pandas as pd

[easydl] tensorflow not available!


In [ ]:
ViT_SKIN?

In [2]:
cscc = ViT_SKIN(train=False,r=4,norm=False,fold=0,flatten=False,adj=True,ori=True,prune='NA')

['P2_ST_rep1']
Loading imgs...
Loading metadata...


In [ ]:
get_cnt method creates

In [3]:
cnt = pd.read_csv('./data/GSE144240_RAW/GSM4284316_P2_ST_rep1_stdata.tsv',sep='\t',index_col=0)
cnt

,MIR1302-2,RP11-34P13.7,RP11-34P13.14,FO538757.1,RP4-669L17.10,RP11-206L10.9,LINC00115,RP11-54O7.1,SAMD11,NOC2L,...,DAZ2,AC012005.2,AC012005.1,BPY2B,DAZ3,DAZ4,BPY2C,AC006386.1,LINC00266-4P,AC006328.1
10x10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10x12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10x14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10x16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10x18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9x57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9x59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9x61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9x7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


this should be the count matrix: rows are spot IDs and columns genes of all the spots

get_pos method

In [4]:
pos_df = pd.read_csv("./data/GSE144240_RAW/GSM4284316_spot_data-selection-P2_ST_rep1.tsv",sep='\t')

x = pos_df['x'].values
y = pos_df['y'].values
x = np.around(x).astype(int)
y = np.around(y).astype(int)
id = []
for i in range(len(x)):
    id.append(str(x[i])+'x'+str(y[i])) 
pos_df['id'] = id
pos_df

,x,y,new_x,new_y,pixel_x,pixel_y,id
0,7,45,6.98,44.94,3282.6,9536.7,7x45
1,8,30,8.04,29.98,3473.4,6539.5,8x30
2,8,32,8.03,31.97,3471.9,6937.9,8x32
3,8,34,8.04,33.97,3474.1,7338.8,8x34
4,8,36,8.01,35.96,3469.5,7736.4,8x36
...,...,...,...,...,...,...,...
661,49,41,48.90,41.06,10850.2,8757.7,49x41
662,49,43,48.97,43.04,10861.3,9155.9,49x43
663,49,45,48.91,45.05,10850.9,9557.6,49x45
664,50,42,49.98,42.05,11044.2,8956.9,50x42


the above appear to only contain data about filtered spots

get_meta creates

In [5]:
meta = cnt.join(pos_df.set_index("id"),how="inner")
meta

,MIR1302-2,RP11-34P13.7,RP11-34P13.14,FO538757.1,RP4-669L17.10,RP11-206L10.9,LINC00115,RP11-54O7.1,SAMD11,NOC2L,...,BPY2C,AC006386.1,LINC00266-4P,AC006328.1,x,y,new_x,new_y,pixel_x,pixel_y
10x26,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,3.0,0.0,0.0,10,26,10.10,26.00,3845.3,5741.5
10x28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,3.0,0.0,0.0,10,28,10.07,28.00,3841.4,6142.9
10x30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,10,30,10.04,30.03,3834.4,6549.5
10x32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,3.0,0.0,0.0,10,32,10.06,32.04,3839.6,6951.6
10x34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,6.0,0.0,0.0,10,34,10.06,34.06,3839.4,7355.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9x41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,9,41,9.01,40.97,3648.8,8741.4
9x43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,9,43,9.02,42.97,3650.9,9141.0
9x45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9,45,8.97,44.97,3642.4,9541.6
9x47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,9,47,8.97,46.96,3642.1,9940.0


In [7]:
meta.x.min(), meta.x.max(), meta.y.min(),meta.y.max()

(7, 50, 12, 56)

__only x,y, and the pixel_x,y versions are needed as positions__ 
caution: the x and y are also the IDs here

I have at my disposal:

### construction of get image
.tiff images that I manually compressed to jpg using imagej. when loaded, they are in the same shape

In [1]:
import glob
glob.glob('./data/GSE240429_RAW/'+'*'+'C73A1'+'_Merged.jpg')[0]

'./data/GSE240429_RAW/GSM7697868_GEX_C73A1_Merged.jpg'

In [4]:
im = Image.open(glob.glob('./data/GSE240429_RAW/'+'*'+'C73A1'+'_Merged.jpg')[0])

In [5]:
im = np.array(im)

In [6]:
im = torch.from_numpy(im) 

In [7]:
im

tensor([[[208, 224, 250],
         [212, 228, 254],
         [215, 231, 255],
         ...,
         [207, 228, 255],
         [205, 226, 253],
         [204, 225, 252]],

        [[211, 227, 253],
         [213, 229, 255],
         [215, 231, 255],
         ...,
         [208, 229, 255],
         [207, 228, 255],
         [206, 227, 254]],

        [[214, 230, 255],
         [215, 231, 255],
         [214, 232, 255],
         ...,
         [209, 231, 255],
         [208, 229, 255],
         [208, 229, 255]],

        ...,

        [[214, 232, 255],
         [213, 231, 255],
         [212, 230, 254],
         ...,
         [213, 230, 255],
         [213, 230, 255],
         [213, 230, 255]],

        [[214, 232, 255],
         [213, 231, 255],
         [212, 230, 254],
         ...,
         [213, 230, 255],
         [214, 231, 255],
         [214, 231, 255]],

        [[214, 232, 255],
         [214, 232, 255],
         [212, 230, 254],
         ...,
         [213, 230, 255],
        

### construction of count matrix

In [6]:
f = pd.read_csv('./data/GSE240429_RAW/GSM7697868_C73A1_features.tsv',sep='\t',header=None)
f

,0,1,2
0,ENSG00000243485,MIR1302-2HG,Gene Expression
1,ENSG00000237613,FAM138A,Gene Expression
2,ENSG00000186092,OR4F5,Gene Expression
3,ENSG00000238009,AL627309.1,Gene Expression
4,ENSG00000239945,AL627309.3,Gene Expression
...,...,...,...
36596,ENSG00000277836,AC141272.1,Gene Expression
36597,ENSG00000278633,AC023491.2,Gene Expression
36598,ENSG00000276017,AC007325.1,Gene Expression
36599,ENSG00000278817,AC007325.4,Gene Expression


In [ ]:
        #barcode_tsv
            #spot barcodes - alphabetical order

In [7]:
bc= pd.read_csv('./data/GSE240429_RAW/GSM7697868_C73A1_barcodes.tsv',sep='\t',header=None)
bc

,0
0,AAACAACGAATAGTTC-1
1,AAACAAGTATCTCCCA-1
2,AAACAATCTACTAGCA-1
3,AAACACCAATAACTGC-1
4,AAACAGAGCGACTCCT-1
...,...
4987,TTGTTTCACATCCAGG-1
4988,TTGTTTCATTAGTCTA-1
4989,TTGTTTCCATACAACT-1
4990,TTGTTTGTATTACACG-1


In [3]:
import scipy.io as sio

In [4]:
mtx = sio.mmread("./data/GSE240429_RAW/GSM7697868_C73A1_matrix.mtx").toarray()

In [30]:
mtx.shape

(36601, 4992)

In [8]:
c=pd.DataFrame(data = mtx, index = f[1], columns = bc[0]).T
c.rename_axis(None, axis=0, inplace=True)
c.rename_axis(None, axis=1, inplace=True)
c

,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,AL627309.4,AP006222.2,AL732372.1,...,AC133551.1,AC136612.1,AC136616.1,AC136616.3,AC136616.2,AC141272.1,AC023491.2,AC007325.1,AC007325.4,AC007325.2
AAACAACGAATAGTTC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAAGTATCTCCCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAATCTACTAGCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACACCAATAACTGC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAGAGCGACTCCT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTTTCATTAGTCTA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTTTCCATACAACT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTTTGTATTACACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I have to add the genes that were not measures, but are part of the genes to be predicted (from KeyError: "['WHSC1L1', 'AC013461.1', 'MLLT4'] not in index")

In [9]:
c['WHSC1L1'] = 0
c["AC013461.1"] = 0
c["MLLT4"] = 0
c

,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,AL627309.4,AP006222.2,AL732372.1,...,AC136616.3,AC136616.2,AC141272.1,AC023491.2,AC007325.1,AC007325.4,AC007325.2,WHSC1L1,AC013461.1,MLLT4
AAACAACGAATAGTTC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAAGTATCTCCCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAATCTACTAGCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACACCAATAACTGC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACAGAGCGACTCCT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTTCACATCCAGG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTTTCATTAGTCTA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTTTCCATACAACT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTTTGTATTACACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


get_cnt: combine

In [ ]:
#import scipy.io as sio

#features_path = glob.glob(self.dir+'*'+name+'_features.tsv')[0]
#barcodes_path = glob.glob(self.dir+'*'+name+'_barcodes.tsv')[0]
#mtx_path = glob.glob(self.dir+'*'+name+'_matrix.mtx')[0]

#f = pd.read_csv(features_path,sep='\t',header=None)
#bc= pd.read_csv(barcodes_path,sep='\t',header=None)
#mtx = sio.mmread(mtx_path).toarray()

#c=pd.DataFrame(data = mtx, index = f[1], columns = bc[0]).T
#c.rename_axis(None, axis=0, inplace=True) #get rid of names
#c.rename_axis(None, axis=1, inplace=True)
#c['WHSC1L1'] = 0
#c["AC013461.1"] = 0
#c["MLLT4"] = 0

#return c

### unused
the following is not used by bleep, and will also not be needed  for hist2st

In [29]:
pd.read_csv('./data/GSE240429_RAW/GSM7697868_C73A1_metrics_summary.csv',index_col=0)

,Number of Spots Under Tissue,Median Genes per Spot,Number of Reads,Valid Barcodes,Valid UMIs,Mean Reads per Spot,Sequencing Saturation,Q30 Bases in Barcode,Q30 Bases in RNA Read,Q30 Bases in UMI,...,Reads Mapped Confidently to Genome,Reads Mapped Confidently to Intergenic Regions,Reads Mapped Confidently to Intronic Regions,Reads Mapped Confidently to Exonic Regions,Reads Mapped Confidently to Transcriptome,Reads Mapped Antisense to Gene,Fraction Reads in Spots Under Tissue,Total Genes Detected,Median UMI Counts per Spot,Fraction of Spots Under Tissue
Sample ID,,,,,,,,,,,,,,,,,,,,,
MacParland_Catia__C73_A1_VIS,2378,2227.0,172491290.0,0.966483,0.996887,72536.286796,0.745478,0.97218,0.939864,0.972224,...,0.849891,0.035695,0.016404,0.797792,0.770266,0.010442,0.794819,20412,8134.0,0.476362


### construction of get_pos
#spatial_pos_csv
            #barcode name
            #detected tissue boolean
            #x spot index
            #y spot index
            #x spot position (px)
            #y spot position (px)

In [12]:
s = pd.read_csv('./data/GSE240429_RAW/GSM7697868_C73A1_tissue_positions_list.csv', header=None)
s

,0,1,2,3,4,5
0,ACGCCTGACACGCGCT-1,0,0,0,1791,2239
1,TACCGATCCAACACTT-1,0,1,1,2078,2403
2,ATTAAAGCGGACGAGC-1,0,0,2,1792,2568
3,GATAAGGGACGATTAG-1,0,1,3,2078,2732
4,GTGCAAATCACCAATA-1,0,0,4,1793,2897
...,...,...,...,...,...,...
4987,GAACGTTTGTATCCAC-1,0,77,123,23874,22403
4988,ATTGAATTCCCTGTAG-1,0,76,124,23588,22568
4989,TACCTCACCAATTGTA-1,0,77,125,23874,22732
4990,AGTCGAATTAGCGTAA-1,0,76,126,23589,22897


column 1 has an integer displaying whether tissue was detected in that spot

In [14]:
s = s.loc[s[1]==1]
s

,0,1,2,3,4,5
1095,GTGCACGAAAGTGACT-1,1,17,71,6686,13900
1103,GTAGGTGATCCGTGTA-1,1,17,79,6689,15215
1105,CTGGCGACATAAGTCC-1,1,17,81,6690,15544
1111,GCGCTGATCCAGACTC-1,1,17,87,6693,16530
1113,GATATTTCCTACATGG-1,1,17,89,6694,16859
...,...,...,...,...,...,...
4592,CTTAACTTACAGTATA-1,1,70,112,21866,20600
4593,GTGAGTCGACTAATAG-1,1,71,113,22153,20763
4594,TGTACTTCCGGGCATG-1,1,70,114,21867,20929
4718,GTCTCCCGAGTCCCGT-1,1,72,110,22437,20269


In [21]:
s= s.rename(columns={0: "id", 1:"tissue", 2:"x", 3:"y", 4:"pixel_x", 5:"pixel_y"}).drop(columns="tissue")
s

,id,x,y,pixel_x,pixel_y
1095,GTGCACGAAAGTGACT-1,17,71,6686,13900
1103,GTAGGTGATCCGTGTA-1,17,79,6689,15215
1105,CTGGCGACATAAGTCC-1,17,81,6690,15544
1111,GCGCTGATCCAGACTC-1,17,87,6693,16530
1113,GATATTTCCTACATGG-1,17,89,6694,16859
...,...,...,...,...,...
4592,CTTAACTTACAGTATA-1,70,112,21866,20600
4593,GTGAGTCGACTAATAG-1,71,113,22153,20763
4594,TGTACTTCCGGGCATG-1,70,114,21867,20929
4718,GTCTCCCGAGTCCCGT-1,72,110,22437,20269


In [22]:
s.dtypes

id         object
x           int64
y           int64
pixel_x     int64
pixel_y     int64
dtype: object

get_pos: combined:

In [ ]:
#path = glob.glob(self.dir+"*"+name+"*tissue_position_list.csv")[0]
#df = pd.read_csv(path, header=None)
#df.loc[df[1]==1] #get the spots covering tissue
#df = df.rename(columns={0: "id", 1:"tissue", 2:"x", 3:"y", 4:"pixel_x", 5:"pixel_y"}).drop(columns="tissue")
#return df